In [ ]:
import os

import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import sklearn.utils as sku

In [ ]:
WORD_DATA_PATH = "./herbier/data_public/words/"
METADATA_PATH = "./herbier/data_public/ascii/words.txt"

IMAGE_HEIGHT = IMAGE_WIDTH = 128

DEBUG = True

## TODO
- voir pour ignorer le fichier METADATA
- limiter le nombre de sample à sélectionner

In [ ]:
def load_words_data(data_path, metadata_path):
    data = []

    with open(metadata_path, 'r') as file:
        for line in file:
            if not line.startswith("#"):
                components = line.strip().split(' ')
                word_id = components[0]
                
                parts = word_id.split('-')
                writer_id = '-'.join(parts[:2])  # e.g., 'a01-000u'
                image_subfolder = parts[0]       # e.g., 'a01'
                image_filename = f"{word_id}.png"
                image_path = os.path.join(data_path, image_subfolder, writer_id, image_filename)
                
                if os.path.exists(image_path):
                    try:
                        img = tf.io.read_file(image_path)
                        img = tf.image.decode_png(img)
                        data.append({
                            'image_path': image_path,
                            'writer_id': writer_id,  # Label = writer ID
                            'image_array': img
                        })
                    except tf.errors.InvalidArgumentError:
                        print(f"Image not found for word ID: {word_id} at {image_path}")
                else:
                    print(f"Image not found for word ID: {word_id} at {image_path}")

    return data

words_data = load_words_data(WORD_DATA_PATH, METADATA_PATH)

if DEBUG:
  print(f"Loaded {len(words_data)} words.")
  for entry in words_data[:5]:
      print(f"  Writer ID: {entry['writer_id']}; image shape: {entry['image_array'].shape}")

if DEBUG: 
    print("number of writers: ", len(set([entry['writer_id'] for entry in words_data])))

if DEBUG:
  plt.figure(figsize=(10, 10))
  for i in range(25):
      plt.subplot(5, 5, i + 1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(words_data[i]['image_array'], cmap=plt.cm.binary)
      plt.xlabel(words_data[i]['writer_id'])
  plt.show()


In [ ]:
# preprocessing data

def preprocess_data(data):
    labels = []
    images = []

    for entry in data:
        # Resize the image
        img = tf.image.resize(entry['image_array'], [IMAGE_HEIGHT, IMAGE_WIDTH])

        img = img.numpy().astype('float32') / 255.0  # Normalize and convert to float32
        images.append(img)

        labels.append(entry['writer_id'])

    return np.array(images), np.array(labels)


images, labels = preprocess_data(words_data)

X_train, X_test, y_train, y_test = skms.train_test_split(images, labels, test_size=0.2, random_state=42)

if DEBUG:
    print(f"X_train: {X_train.shape}; y_train: {y_train.shape}")
    print(f"X_test: {X_test.shape}; y_test: {y_test.shape}")

In [ ]:
# encode labels

label_encoder = skp.LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)
one_hot_encoded_labels = keras.utils.to_categorical(integer_encoded_labels)

X_train, X_test, y_train, y_test = skms.train_test_split(images, one_hot_encoded_labels, test_size=0.2, random_state=42)

if DEBUG:
    print(f"X_train: {X_train.shape}; y_train: {y_train.shape}")
    print(f"X_test: {X_test.shape}; y_test: {y_test.shape}")